# Receive output from LR generator for a high resoultion image

# Imports

In [ ]:
import torch
import torch.nn as nn

from PIL import Image
from torchvision.transforms import ToTensor, ToPILImage, Normalize

# LR Generator class

In [ ]:
class GLR(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super(GLR, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.2),
             self._block(64, 128, 3, 2),
             self._block(128, 256, 3, 2),
            nn.Conv2d(256, out_channels, kernel_size=3, padding=1)
        )

    def _block(self, in_channels, out_channels, kernel_size, stride):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2)
        )

    def forward(self, x):
        return self.net(x)

## Load the weights of the pre-trained model

In [ ]:
# Model, Optimizer, and Losses
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Assume GSR is the class definition for the SR generator
generator_lr = GLR().to(device)  # Make sure the device is set (e.g., 'cuda' or 'cpu')

# Load the pre-trained weights
generator_lr.load_state_dict(torch.load('generator_lr.pth'))

# Set the model to evaluation mode
generator_lr.eval()

GLR(
  (net): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (3): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.2)
    )
    (4): Conv2d(256, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
)

## Prepare the high-resolution image

In [ ]:
# Load the low-resolution image
hr_image = Image.open('input/HR/8e5ce3b37f7e73.jpg').convert('RGB')

# Preprocess the image: Convert to tensor and add a batch dimension
transform = ToTensor()
hr_image_tensor = transform(hr_image).unsqueeze(0).to(device)  # Add batch dimension and move to device

## Generate the low-resolution image

In [ ]:
# Generate the high-resolution image
with torch.no_grad():  # Disable gradient calculation for inference
    lr_image_tensor = generator_lr(hr_image_tensor)

# Remove the batch dimension and convert back to a PIL image
lr_image_tensor = lr_image_tensor.squeeze(0)  # Remove batch dimension
to_pil = ToPILImage()
lr_image = to_pil(lr_image_tensor.cpu())  # Move to CPU and convert to PIL image

## Save low resolution image output

In [ ]:
# Save the high-resolution image
lr_image.save('output/LR/8e5ce3b37f7e73.jpg')